# Web Crawing
 : 웹페이지 데이터 수집

## 크롤링 방법

##### 크롤링 정책
    - robots.txt 파일에 크롤링 정책 설명 : https://www.해당 주소.com/robots.txt
    - 크롤링에 대한 법적 제제 없음
    - 과도한 크롤링으로 서비스에 영향을 주는 경우 문제가 될 수 있음
        - 영업방해, 지적재산권 침해 등 다른 법률에 의해 제재
            ex) 사람인/잡코리아(2008 ~ 2018)
    - 크롤링을 할때는 되도록 api를 사용

- requests package
    - json 문자열 파싱 (동적 크롤링에 주로 사용)
    - html 문자열 파싱 (정적 크롤링에 주로 사용)

In [ ]:
import requests, json
import pandas as pd

##### 1. 실습 ( 웹서비스 분석 : 크롬 개발자 도구 url )
 - 코스피 url : https://m.stock.naver.com/api/index/KOSPI/price?pageSize=10&page=3

In [ ]:
url = 'https://m.stock.naver.com/api/index/KOSPI/price?pageSize=10&page=3'
response = requests.get(url)

print(response) # <Response [200]> : response status

In [ ]:
# request -> string (response)
print(type(response.text)) # string
print()
print(response.text)
print()
print(response.text[:200])

In [ ]:
# response -> list (data)
data = response.json()
print(type(data)) # list
print()
print(data)

In [ ]:
# data -> DataFrame (df)
df = pd.DataFrame(data)[['localTradedAt', 'closePrice']]
print(df.dtypes)
print()
print(df)

In [ ]:
# 함수 생성
def stock_crawling(pagesize, page, code='KOSPI'):
    """stock page crawling

    Args:
        pagesize (int): page size
        page (int): page number
        code (str, optional): KOSPI / KOSDAQ. Defaults to 'KOSPI'.

    Returns:
        DataFrame : 'localTradedAt', 'closePrice'
    """
    
    url = f'https://m.stock.naver.com/api/index/{code}/price?pageSize={pagesize}&page={page}'
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data)[['localTradedAt', 'closePrice']]
    
    return df


 - 코스닥 url : https://m.stock.naver.com/api/index/KOSDAQ/price?pageSize=10&page=2

In [ ]:
url = 'https://m.stock.naver.com/api/index/KOSDAQ/price?pageSize=10&page=2'
response = requests.get(url)
data = response.json()
df = pd.DataFrame(data)[["localTradedAt", "closePrice"]]

print(df)

print(stock_crawling(10,2,'KOSDAQ'))

 - 환율 ( 달러 ) url : https://api.stock.naver.com/marketindex/exchange/FX_USDKRW/prices?page=3&pageSize=10

In [ ]:
url = 'https://api.stock.naver.com/marketindex/exchange/FX_USDKRW/prices?page=3&pageSize=10'
response = requests.get(url)
data = response.json()
df = pd.DataFrame(data)[["localTradedAt", "closePrice"]]

print(df)

def exchange_crawling(pagesize, page, code='FX_USDKRW'):
    """money exchange

    Args:
        pagesize (int): page size
        page (int): page number
        code (str, optional): FX_USDKRW, FX_EURKRW, FX_JPYKRW, FX_CNYKRW, each. Defaults to 'FX_USDKRW'.

    Returns:
        _DataFrame : 'localTradedAt', 'closePrice'
    """
    
    url = f'https://api.stock.naver.com/marketindex/exchange/{code}/prices?page={page}&pageSize={pagesize}'
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data)[['localTradedAt', 'closePrice']]
    
    return df

print(exchange_crawling(10,1))

### 데이터 분석 : 상관관계
 - 피어슨 상관계수
    - 1과 가까울수록 강한 양의 상관관계를 갖는다.
    - -1과 가까울수록 강한 음의 상관계를 갖는다.
    - 0과 가까울수록 관계가 없다.
    - pandas 함수 df.corr()

ex) 달러 환율과 주가 지수의 관계

In [ ]:
kospi = stock_crawling(60, 1)
kosdaq = stock_crawling(60, 1, 'KOSDAQ')
usd = exchange_crawling(60,1)

df = kospi.copy()
df['kosdaq'] = kosdaq["closePrice"]
df['usd'] = usd["closePrice"]
df.rename(columns={'closePrice': 'kospi'},
          inplace=True)

print(df.dtypes)
print()
# 데이터타입 변경 str -> int
df['kospi'] = df['kospi'].apply(lambda data:float(data.replace(',','')))
df['kosdaq'] = df['kosdaq'].apply(lambda data:float(data.replace(',','')))
df['usd'] = df['usd'].apply(lambda data:float(data.replace(',','')))
print(df.dtypes)
print()

print(df[['kospi', 'kosdaq', 'usd']].corr())



In [ ]:
# apply(func)
df = pd.DataFrame([{"age": 23}, {"age": 36}, {"age": 27}])

# 연령대 컬럼을 추가 
def change_ages(age): 
    return age // 10 * 10

df["ages"] = df["age"].apply(change_ages)
print(df)
print()

In [ ]:
### lambda : 일회성 함수
###     - 사용 이유 : 메모리 절약, 간단한 함수 (파라미터를 받아서 바로 리턴)

# 함수 3개 사용 ( 메모리 3개 사용 )
def plus(n1,n2):
    return n1+n2

def minus(n1,n2):
    return n1-n2

def calc(func, n1, n2):
    return func(n1, n2)

print('def : plus, minus, calc')
print(calc(plus, 1,2))
print(calc(minus, 1,2))
print()

# lambda parmeter : return
print('lambda : calc')
print(calc(lambda n1, n2: n1+n2, 1, 2))
print(calc(lambda n1, n2: n1-n2, 1, 2))


### API 데이터 수집
 - application programing interface
 - api를 사용해서 데이터를 수집하는 것은 서비스에 데이터를 제공하는 공식적인 방법으로 데이터 수집

ex) naver api( 파파고 )

In [ ]:
import pandas as pd
import requests, json

# 1. app 등록 > app_key(request_token)
# https://developers.naver.com
CLIENT_ID, CLIENT_SECRET = "Y7kkW5s3waqiGemisIDm", "O0iVtJhKyp"

# 2. Naver API document 확인 > URL
# https://openapi.naver.com/v1/papago/n2mt
url = 'https://openapi.naver.com/v1/papago/n2mt'
txt = '안녕하세요. 반갑습니다. 감사합니다.'
params = {
    'source': 'ko',
    'target': 'en',
    'text':txt
}
headers = {
    "Content-Type": "application/json",
    "X-Naver-Client-Id": CLIENT_ID,
    "X-Naver-Client-Secret": CLIENT_SECRET
}
# 3. request(url, app_key) > json(str)
ppg_response = requests.post(url, json.dumps(params), headers=headers)
print(ppg_response)
print()

# 4. json > list, dict > dataframe
txt_en = ppg_response.json()["message"]["result"]["translatedText"]
print(txt_en)

In [ ]:
# 파파고 함수
def ppg_translate(id,pw,txt,fromNa='ko', toNa='en'):
    """naver papago translate api

    Args:
        id (str): naver api id
        pw (str): naver api secret_
        txt (str): befor translate text
        fromNa (str, optional): . Defaults to 'ko'.
        toNa (str, optional): . Defaults to 'en'.

    Returns:
        _type_: DataFrame, after translate text
    """
    
    params = {
        'source': fromNa,
        'target': toNa,
        'text':txt
    }
    
    headers = {
        "Content-Type": "application/json",
        "X-Naver-Client-Id": id,
        "X-Naver-Client-Secret": pw
    }
    response = requests.post('https://openapi.naver.com/v1/papago/n2mt', json.dumps(params), headers=headers)
    
    return response.json()["message"]["result"]["translatedText"]

print(ppg_translate(CLIENT_ID, CLIENT_SECRET, '크롤링 공부'))

In [ ]:
### covid 번역
CLIENT_ID, CLIENT_SECRET = "Y7kkW5s3waqiGemisIDm", "O0iVtJhKyp"

def ppg_translate_txt(txt):
    """naver papago translate api

    Args:
        txt (str): befor translate text

    Returns:
        _type_: DataFrame, after translate text
    """
    
    params = {
        'source': 'ko',
        'target': 'en',
        'text':txt
    }
    headers = {
        "Content-Type": "application/json",
        "X-Naver-Client-Id": CLIENT_ID,
        "X-Naver-Client-Secret": CLIENT_SECRET
    }
    
    response = requests.post('https://openapi.naver.com/v1/papago/n2mt', json.dumps(params), headers=headers)
    
    return response.json()["message"]["result"]["translatedText"]

covid = pd.read_excel("../excel/covid.xlsx")[["category", "title"]]
covid['en'] = covid['title'].apply(ppg_translate_txt)
print(covid)
# 엑셀 파일 만들기
# covid.to_excel('../excel/covid_en.xlsx', index=False, encoding='utf-8-sig')

ex) naver dataLab

In [ ]:
# datalab.naver.com
import pandas as pd
import requests, json

id, secret = 'Y7kkW5s3waqiGemisIDm', 'O0iVtJhKyp'
url = 'https://openapi.naver.com/v1/datalab/search'
headers = {
    'X-Naver-Client-Id': id,
    'X-Naver-Client-Secret':secret,
    'Content-Type': 'application/json'
}

data = {
    "startDate": "2018-01-01",
    "endDate": "2022-08-04",
    "timeUnit": "month",
    "keywordGroups": [
                    {
                        "groupName": "트위터",
                        "keywords": ["트위터", "트윗",'twitter']
                    },
                    {
                        "groupName": "페이스북",
                        "keywords": ["페이스북","페북", 'facebook']
                    },
                    {
                        "groupName": "인스타그램",
                        "keywords": ["인스타그램","인스타", 'insta', 'instagram']
                    }
                    ]
}
dtLab = requests.post(url=url, data=json.dumps(data) ,headers=headers)

print(dtLab)
# print(dtLab.text)


In [ ]:
dt = dtLab.json()["results"]
print([i['ratio'] for i in dt[0]['data']])
print(type(data['keywordGroups'][0]['keywords'][-1]))
print()

df = pd.DataFrame(dt[0]['data'])['period']
# print(period)
for i in range(len(data['keywordGroups'])):
    df_ = pd.DataFrame(dt[i]['data'])['ratio']
    df = pd.concat([df,df_], axis=1)
    
    df.rename(columns={'ratio':data['keywordGroups'][i]['keywords'][-1]}
              , inplace=True)
    

print(df)

# df = pd.DataFrame({
#     "date": [period["period"] for period in dt[0]["data"]],
#     "twitter": [ratio["ratio"] for ratio in dt[0]["data"]],
#     "facebook": [ratio["ratio"] for ratio in dt[1]["data"]],
#     "instagram": [ratio["ratio"] for ratio in dt[2]["data"]],
# })


In [ ]:
# visualizations
df.plot(figsize=(20, 5))

##### 직방 매물정보 수집
1. 지역 검색
2. 위도, 경도 확인
3. 매물 아이디 확인
4. 매물 정보 확인

In [ ]:
# 위도,경도 hash 패키지
import geohash2

# 1. 지역 검색
addr = '주례동'
url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'

response = requests.get(url)
data = response.json()['items'][0]
print(response.json())

lat, lng = data['lat'],data['lng']

# print(response.text)
# print()
# print(data)
# print()
# print(lat,lng)

In [ ]:
# 2. 위도, 경도 확인
geohash = geohash2.encode(lat, lng, precision=5)

print(geohash)

In [ ]:
# 3. 매물 아이디 확인
itm_url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'

itm_respon = requests.get(itm_url)
print(itm_respon)

items = itm_respon.json()["items"] 
print(len(items), items[:2])
print()

ids = [item['item_id'] for item in items]
print(ids)

In [ ]:
# 4. 매물 정보 확인
room_url = 'https://apis.zigbang.com/v2/items/list'
params = {
    'domain': 'zigbang',
    'item_ids':ids,
    'withCoalition': 'true'
}

room_respon = requests.post(url=room_url, params=params)

room = room_respon.json()['items']
columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "address1", "manage_cost"]

df = pd.DataFrame(room)[columns]
print(df)

In [219]:
# max row, max column 설정
print(pd.options.display.max_columns, pd.options.display.max_rows)
pd.options.display.max_rows = 200

50 60


In [ ]:
# 직방 함수
import pandas as pd
import geohash2, json, requests


def zigbang(addr):
    """zigbang room search

    Args:
        addr (str): address

    Returns:
        DataFrame: columns - ["item_id", "sales_type", "deposit", "rent", "size_m2", "address1", "manage_cost"]
    """
    
    url1 = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'
    
    dt1 = requests.get(url1).json()['items'][0]
    lat, lng = dt1['lat'],dt1['lng']
    geohash = geohash2.encode(lat, lng, precision=5)
    
    url2 = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
    
    dt2 = requests.get(url2).json()["items"]
    idx = [item['item_id'] for item in dt2]
    
    url3 = 'https://apis.zigbang.com/v2/items/list'
    
    params = {
        'domain': 'zigbang',
        'item_ids':idx,
        'withCoalition': 'true'
    }
    dt3 = requests.post(url=url3, params=params).json()['items']
    columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "address1", "manage_cost"]
    df = pd.DataFrame(dt3)[columns]
    
    
    return df

print(zigbang('주례동'))

### html
 - 웹 문서를 작성하는 언어
 - 구성요소
    - Document : 한페이지를 나타내는 코드
    - Element : 하나의 레이아웃, Element가 모여서 Document를 만든다.
    - Tag : Element의 종류 정의, 시작/끝 태그가 모여서 Element를 만든다.
    - Attribute : Tag의 속성. 시작 태그에서 태그의 기능을 정의
        - id : 페이지 내 유일한 값
        - class : 페이지 내 중복 가능한 값
        - attr : 속성값
    - Text : 시작과 끝 태그 사이의 내용
 - Element는 계층적 구조를 갖는다.
 



%%html

<div class='wrapper'>
    <button id='b1' clas='btn' value='1'>html1</button>
    <button id='b2' clas='btn' value='2'>html1</button>
</div>


In [ ]:
# 파이썬 부동 소숫점 오류
a, b = 0.1, 0.2
a+b # 0.3 x

In [ ]:
(a+b) == 0.3

In [ ]:
# 해결 : 반올림 사용
round(a+b,1) == 0.3

In [ ]:
# 해결 : 고정소수점 연산
from decimal import Decimal
float(Decimal(str(a)) + Decimal(str(b))) == 0.3


### 정적페이지 데이터 수집
 - 네이버 연관 검색어 수집
 - bs4 : BeautifulSoup, HTML > CSS selector 를 이용하여 수집

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# 1. 웹 서비스 분석 : url
url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query=kt&oquery=%EC%A1%B4%EB%B0%94%EB%B0%94%ED%86%A0%EC%8A%A4&tqi=hXWxYsprvmsss79TtHossssstaN-185816'

response = requests.get(url)
print(response)

In [ ]:
doc = BeautifulSoup(response.text, 'html.parser')

elements = doc.select('.lst_related_srch > .item')
elements

In [ ]:
dt = [e.select_one('.tit').text for e in elements]
dt

### Gmarket 베트스 상품 수집

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

url = 'http://corners.gmarket.co.kr/Bestsellers'

response = requests.get(url)
response

In [221]:
dt = BeautifulSoup(response.text, 'html.parser')
dt

elements = dt.select('#gBestWrap > div > div:nth-child(5) > div > ul > li')

items = []
for e in elements:
    item = {
        'title': e.select_one('.itemname').text
        , 'o-price': e.select_one('.o-price').text.replace(',','').replace('원','')
        , 's-price': e.select_one('.s-price > strong').text.replace(',','').replace('원','')
        , 'img': 'http:' + e.select_one('img').get('data-original')
    }
    
    items.append(item)
    
df = pd.DataFrame(items)


def typeInt(data):
    if data == '':
        return 0
    else:
        return int(data)
    
    

# df['o-price'] = df['o-price'].apply(typeInt(data))
df['s-price'] = df['s-price'].apply(lambda data:int(data))


df[['o-price', 's-price']]

TypeError: int() argument must be a string, a bytes-like object or a number, not 'dict'